In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("output.csv")

In [5]:
x = df.drop("num_orders", axis=1)
y = df['num_orders']

In [6]:
import evalml

In [7]:
evalml.problem_types.ProblemTypes.all_problem_types

[<ProblemTypes.BINARY: 'binary'>,
 <ProblemTypes.MULTICLASS: 'multiclass'>,
 <ProblemTypes.REGRESSION: 'regression'>,
 <ProblemTypes.TIME_SERIES_REGRESSION: 'time series regression'>,
 <ProblemTypes.TIME_SERIES_BINARY: 'time series binary'>,
 <ProblemTypes.TIME_SERIES_MULTICLASS: 'time series multiclass'>]

In [8]:
X_train,X_test,y_train,y_test=evalml.preprocessing.split_data(x,y,problem_type='regression')

In [9]:
from evalml import AutoMLSearch

In [10]:
automl=AutoMLSearch(X_train=X_train,y_train=y_train,problem_type='regression',max_batches=1,optimize_thresholds=True)

Generating pipelines to search over...


In [11]:
automl.search()


*****************************
* Beginning pipeline search *
*****************************

Optimizing for R2. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: decision_tree, xgboost, linear_model, extra_trees, lightgbm, random_forest, catboost



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mean Baseline Regression Pipeline
Mean Baseline Regression Pipeline:
	Starting cross validation
	Finished cross validation - mean R2: -0.000

*****************************
* Evaluating Batch Number 1 *
*****************************

Linear Regressor w/ Imputer + One Hot Encoder + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean R2: 0.638
Decision Tree Regressor w/ Imputer + One Hot Encoder:
	Starting cross validation
	Finished cross validation - mean R2: 0.578
Random Forest Regressor w/ Imputer + One Hot Encoder:
	Starting cross validation
	Finished cross validation - mean R2: 0.606
LightGBM Regressor w/ Imputer + One Hot Encoder:
	Starting cross validation
	Finished cross validation - mean R2: 0.680
Elastic Net Regressor w/ Imputer + One Hot Encoder + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean R2: 0.152
XGBoost Regressor w/ Imputer + One Hot Encoder:
	Starting cross validation
	Finished cross

In [12]:
automl.rankings

,id,pipeline_name,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,6,XGBoost Regressor w/ Imputer + One Hot Encoder,0.796280,NaN,0.796280,3.416946e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,4,LightGBM Regressor w/ Imputer + One Hot Encoder,0.679586,NaN,0.679586,2.916209e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,1,Linear Regressor w/ Imputer + One Hot Encoder ...,0.638347,NaN,0.638347,2.739252e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,3,Random Forest Regressor w/ Imputer + One Hot E...,0.606021,NaN,0.606021,2.600541e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,7,Extra Trees Regressor w/ Imputer + One Hot Enc...,0.601293,NaN,0.601293,2.580252e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,2,Decision Tree Regressor w/ Imputer + One Hot E...,0.577567,NaN,0.577567,2.478444e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
6,8,CatBoost Regressor w/ Imputer,0.218588,NaN,0.218588,9.380647e+05,False,{'Imputer': {'categorical_impute_strategy': 'm...
7,5,Elastic Net Regressor w/ Imputer + One Hot Enc...,0.152399,NaN,0.152399,6.540472e+05,False,{'Imputer': {'categorical_impute_strategy': 'm...
8,0,Mean Baseline Regression Pipeline,-0.000023,NaN,-0.000023,0.000000e+00,False,{'Baseline Regressor': {'strategy': 'mean'}}


In [13]:
automl.best_pipeline

pipeline = RegressionPipeline(component_graph=[Imputer, OneHotEncoder, XGBoostRegressor], parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'One Hot Encoder':{'top_n': 10, 'features_to_encode': None, 'categories': None, 'drop': 'if_binary', 'handle_unknown': 'ignore', 'handle_missing': 'error'}, 'XGBoost Regressor':{'eta': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 100}}, random_seed=0)

In [14]:
best_pipeline = automl.best_pipeline

In [15]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])


**************************************************
* XGBoost Regressor w/ Imputer + One Hot Encoder *
**************************************************

Problem Type: regression
Model Family: XGBoost

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. One Hot Encoder
	 * top_n : 10
	 * features_to_encode : None
	 * categories : None
	 * drop : if_binary
	 * handle_unknown : ignore
	 * handle_missing : error
3. XGBoost Regressor
	 * eta : 0.1
	 * max_depth : 6
	 * min_child_weight : 1
	 * n_estimators : 100

Training
Training for regression problems.
Total training time (including CV): 19.1 seconds

Cross Validation
----------------
               R2 ExpVariance MaxError MedianAE   MSE   MAE Root Mean Squared Error # Training # Validation
0           0.796       0.796    3.533    0.349 0.303 0.428                   0.550     91,309      273,929
mean        0.796

In [16]:
scores = best_pipeline.score(X_test, y_test,  objectives=evalml.objectives.get_core_objectives('regression'))

In [17]:
print(scores)

OrderedDict([('ExpVariance', 0.7981113454000919), ('MaxError', 3.358474084416724), ('MedianAE', 0.346216236936868), ('MSE', 0.30024308396226296), ('MAE', 0.42575590895537946), ('R2', 0.7981084319203572), ('Root Mean Squared Error', 0.5479444168547235)])


In [18]:
X_train.head()

Data Column,Unnamed: 0,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,category,cuisine,city_code,region_code,center_type,op_area
Physical Type,Int64,Int64,Int64,Int64,Int64,float64,float64,Int64,Int64,category,category,Int64,Int64,category,float64
Logical Type,Integer,Integer,Integer,Integer,Integer,Double,Double,Integer,Integer,Categorical,Categorical,Integer,Integer,Categorical,Double
Semantic Tag(s),['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['category'],['category'],['numeric'],['numeric'],['category'],['numeric']
141016,141016,1473139,48,81,2139,277.45,277.45,0,0,Beverages,Indian,526,34,TYPE_A,4.0
406574,406574,1220479,130,143,1216,414.25,413.25,0,0,Pasta,Italian,562,77,TYPE_B,3.8
189872,189872,1186734,64,146,2581,457.90,639.23,1,0,Pizza,Continental,526,34,TYPE_B,5.0
323934,323934,1209372,105,73,1770,411.28,486.03,0,0,Biryani,Indian,576,34,TYPE_A,4.0
142275,142275,1467845,48,23,2867,620.86,620.86,0,0,Seafood,Continental,698,23,TYPE_A,3.4
